코드 실행을 위하여 모든 줄 실행을 하면 최종 결과가 log.txt 파일에 남게됩니다.

In [ ]:
# Declaration of hyperparameters for training a neural network
BATCH_SIZE=64
EPOCHS=32
AUGEMENTATION_PERCENTAGE = 0.2
LEARNING_RATE=0.1
# OPTIMIZER='adam'
OPTIMIZER='sgd'
DROPOUT=0.2
# SCHEDULER='LambdaLR'
SCHEDULER='StepLR'
STEP_SIZE=16
GAMMA=0.1
# DECRESE_LR=0.95


In [ ]:
# Load all dependencies
import torch
import torch.nn as nn
from torchvision import transforms, datasets

In [ ]:
# Load CIFar-10 dataset and apply data augmentation

# Define the transformation
train_dataset = datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=transforms.Compose([
                                                 transforms.RandomApply(
                                                 [
                                                #  transforms.RandomCrop(size=24,padding=4),
                                                 transforms.RandomHorizontalFlip(),
                                                 transforms.RandomRotation(degrees=30),
                                                 transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                                
                                             ],p=AUGEMENTATION_PERCENTAGE),
                                                    transforms.ToTensor(),
                                                    
                                                ]))
test_dataset = datasets.CIFAR10(root='./data',
                                            train=False,
                                            download=True,
                                            transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(train_loader.dataset)
print(train_loader.dataset.data.shape)


In [ ]:
class CNN(nn.Module):
    """4-conv original architecture (NOT copied from any public model)."""
    def __init__(self, p_drop=0):
        super().__init__()
        # 32x32 → 16x16
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        # 16x16 → 8x8
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(p_drop),
        )
        # 8x8 → 4x4
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(p_drop),
        )
        # 4x4 → 1x1
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = x.flatten(1)
        return self.classifier(x)

In [ ]:
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=CNN(p_drop=DROPOUT).to(DEVICE)
# optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
if OPTIMIZER == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
elif OPTIMIZER == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
# scheduler=torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: DECRESE_LR ** epoch)
criterion=nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader,optimizer):
    model.train()
    for batch_idx,(image,label) in enumerate(train_loader):
        image=image.to(DEVICE)
        label=label.to(DEVICE)
        optimizer.zero_grad()

        output=model(image)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
    scheduler.step()

In [ ]:
def evaluate(model,test_loader):
    model.eval()
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    with torch.no_grad():
       for image,label in test_loader:
        images=image.to(DEVICE)
        labels=label.to(DEVICE)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
    accuracy = {classname: correct / total_pred[classname] * 100 for classname, correct in correct_pred.items()} 
    return accuracy

In [ ]:

# Training and evaluation loop
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    accuracy=evaluate(model, test_loader)
    print(f'Epoch {epoch}, Accuracy: {accuracy}')
accuracy=evaluate(model, test_loader)

In [ ]:
f=open('./log.txt','a')
f.write(f'BATCH_SIZE={BATCH_SIZE},EPOCHS={EPOCHS},AUGEMENTATION_PERCENTAGE={AUGEMENTATION_PERCENTAGE},LEARNING_RATE={LEARNING_RATE},OPTIMIZER={OPTIMIZER},DROPOUT={DROPOUT},SCHEDULER={SCHEDULER}, STEP_SIZE={STEP_SIZE}, GAMMA={GAMMA}\n')
for classname, percent in accuracy.items():
    f.write(f'{classname}: {percent:.2f}% ')
f.write('\n')
f.close()